# --------------------------------------------------------------
# Day 21 NFL.py 🏈: Finalizing dataset
# --------------------------------------------------------------

# Necessary data cleaning

In [1]:
import nfl_data_py as nfl

import pandas as pd
import skimpy
import numpy as np
import requests
from bs4 import BeautifulSoup

import time

In [2]:
years = range(2010, 2023)
data = nfl.import_weekly_data(years)
draft = nfl.import_draft_picks(years)
df = data[(data['season'] >= 2010) & (data['position'] == 'QB')]

rounds = [draft[draft['pfr_player_name'] == plyr]['round'] for plyr in df['player_display_name']]

df_draft = draft[(draft['round'] == 1) & (draft['position'] == 'QB')][['pfr_player_name', 'season', 'round', 'pick', 'team', 'age', 'college']]
df_draft.loc[df_draft['team'] == 'STL', 'team'] = 'LA'

power_5_schools = {
    # ACC (Atlantic Coast Conference)
    'Boston College': True,
    'Clemson': True,
    'Duke': True,
    'Florida St.': True,
    'Georgia Tech': True,
    'Louisville': True,
    'Miami': True,
    'North Carolina': True,
    'NC St.': True,
    'Pittsburgh': True,
    'Syracuse': True,
    'Virginia': True,
    'Virginia Tech': True,
    'Wake Forest': True,
    
    # Big Ten Conference
    'Illinois': True,
    'Indiana': True,
    'Iowa': True,
    'Maryland': True,
    'Michigan': True,
    'Michigan St.': True,
    'Minnesota': True,
    'Nebraska': True,
    'Northwestern': True,
    'Ohio St.': True,
    'Penn St.': True,
    'Purdue': True,
    'Rutgers': True,
    'Wisconsin': True,
    
    # Big 12 Conference
    'Baylor': True,
    'BYU': True,
    'Cincinnati': True,
    'Houston': True,
    'Iowa St.': True,
    'Kansas': True,
    'Kansas St.': True,
    'Oklahoma St.': True,
    'TCU': True,
    'Texas Tech': True,
    'UCF': True,
    'West Virginia': True,
    
    # Pac-12 Conference (as of 2023 realignment)
    'Arizona': True,
    'Arizona St.': True,
    'California': True,
    'Colorado': True,
    'Oregon': True,
    'Oregon St.': True,
    'Stanford': True,
    'UCLA': True,
    'USC': True,
    'Utah': True,
    'Washington': True,
    'Washington St.': True,

    # Southeastern Conference (SEC)
    'Alabama': True,
    'Arkansas': True,
    'Auburn': True,
    'Florida': True,
    'Georgia': True,
    'Kentucky': True,
    'LSU': True,
    'Mississippi St.': True,
    'Missouri': True,
    'Ole Miss': True,
    'South Carolina': True,
    'Tennessee': True,
    'Texas A&M': True,
    'Vanderbilt': True
}

power_5_bool = []
for college in df_draft['college']:
    try:
        power_5_bool.append(power_5_schools[college])
    except:
        power_5_bool.append(False)

df_draft['power_5'] = power_5_bool
    
labels = [df[df['player_display_name'] == plyr] for plyr in df_draft['pfr_player_name']]

players_by_season = df.groupby(['player_display_name', 'season']).agg({'recent_team':lambda x: x.iloc[0]}).reset_index()

labels = {}
for plyr in df_draft['pfr_player_name']:
    draft_team = np.squeeze(df_draft[df_draft['pfr_player_name'] == plyr]['team'])
    draft_year = np.squeeze(df_draft[df_draft['pfr_player_name'] ==plyr]['season'].values[0])
    labels[plyr] = players_by_season[(players_by_season['player_display_name'] == plyr) & (players_by_season['recent_team'] == draft_team)].shape[0] >= 5

Downcasting floats.


# Data scraping

In [3]:
test_df = pd.DataFrame({
    'player':list(labels.keys()),
    'successful':list(labels.values())
})

In [4]:
test_df.loc[test_df['player'] == 'Ryan Tannehill', 'successful'] = True
test_df.loc[test_df['player'] == 'Jared Goff', 'successful'] = True
test_df.loc[test_df['player'] == 'Trevor Lawrence', 'successful'] = True
test_df.loc[test_df['player'] == 'Joe Burrow', 'successful'] = True
test_df.loc[test_df['player'] == 'Tua Tagovailoa', 'successful'] = True
test_df.loc[test_df['player'] == 'Justin Herbert', 'successful'] = True
test_df.loc[test_df['player'] == 'Jordan Love', 'successful'] = True
test_df.loc[test_df['player'] == 'Kyler Murray', 'successful'] = True
test_df.loc[test_df['player'] == 'Baker Mayfield', 'successful'] = True
test_df.loc[test_df['player'] == 'Josh Allen', 'successful'] = True
test_df.loc[test_df['player'] == 'Lamar Jackson', 'successful'] = True
test_df.loc[test_df['player'] == 'Patrick Mahomes', 'successful'] = True
test_df.loc[test_df['player'] == 'Deshaun Watson', 'successful'] = True

In [5]:
print(test_df[test_df['successful'] == True].shape)
print(test_df[test_df['successful'] == False].shape)

(18, 2)
(22, 2)


In [6]:
df_draft['player'] = df_draft['pfr_player_name']
df_draft = df_draft.drop('pfr_player_name', axis=1)
df = pd.merge(test_df, df_draft,on='player', how='inner')

In [7]:
def extract_passing_table(player_name):

    first_name = player_name.split(' ')[0].lower()
    last_name = player_name.split(' ')[1].lower()
    player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-1.html'
    html_content = requests.get(player_url).text
    
    if player_name == 'Zach Wilson':
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-3.html'
        html_content = requests.get(player_url).text
        
    if player_name == 'Justin Fields' or player_name == 'Jordan Love':
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-2.html'
        html_content = requests.get(player_url).text
    
    if player_name == 'Daniel Jones':
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-4.html'
        html_content = requests.get(player_url).text
    
    if player_name == 'Josh Allen':
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-7.html'
        html_content = requests.get(player_url).text

    if player_name == 'Mitchell Trubisky':
        first_name = player_name.split(' ')[0].lower()
        first_name = 'Mitch'
        last_name = player_name.split(' ')[1].lower()
        player_url = 'https://www.sports-reference.com/cfb/players/mitch-trubisky-1.html'
        html_content = requests.get(player_url).text

    if len(player_name.split(' ')) > 2:
        first_name = player_name.split(' ')[0].lower()
        last_name = player_name.split(' ')[1].lower()
        suffix = player_name.split(' ')[2].lower()
        player_url = f'https://www.sports-reference.com/cfb/players/{first_name}-{last_name}-{suffix}-1.html'
        html_content = requests.get(player_url).text
    
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'id': 'passing_standard'})
    
    if not table:
        print("Passing stats table not found")
        return None

    baseline_headers = []
    headers = [th.getText() for th in table.find_all('th')]
    rows = []
    
    for tr in table.find_all('tr')[1:]:
        cells = [td.getText() for td in tr.find_all('td')]
        if cells: 
            rows.append(cells)
    
    yr_college = len(rows) - 1

    column_names = [
    'G',        
    'Cmp',      
    'Att',      
    'Cmp%',     
    'Yds',      
    'TD',       
    'TD%',      
    'Int',      
    'Int%',     
    'Y/A',      
    'AY/A',     
    'Y/C',      
    'Y/G',      
    'Rate',
    'seasons',
    'name'
    ]
    
    totals = rows[len(rows)-1]
    totals = [stat for stat in totals if stat.strip()]
    totals.append(yr_college)
    totals.append(player_name)

    if len(totals) == len(column_names):
        final = pd.DataFrame({name: [value] for name, value in zip(column_names, totals)})
    else:
        print("Error: Number of stats does not match number of column names.")

    return final

In [8]:
player_data = []
for name in df['player']:
    print(f'------Extracting {name} data------')
    player_data.append(extract_passing_table(name))
    time.sleep(5)

------Extracting Sam Bradford data------
------Extracting Tim Tebow data------
------Extracting Cam Newton data------
------Extracting Jake Locker data------
------Extracting Blaine Gabbert data------
------Extracting Christian Ponder data------
------Extracting Andrew Luck data------
------Extracting Robert Griffin III data------
------Extracting Ryan Tannehill data------
------Extracting Brandon Weeden data------
------Extracting EJ Manuel data------
------Extracting Blake Bortles data------
------Extracting Johnny Manziel data------
------Extracting Teddy Bridgewater data------
------Extracting Jameis Winston data------
------Extracting Marcus Mariota data------
------Extracting Jared Goff data------
------Extracting Carson Wentz data------
Passing stats table not found
------Extracting Paxton Lynch data------
------Extracting Kenny Pickett data------
------Extracting Trevor Lawrence data------
------Extracting Zach Wilson data------
------Extracting Trey Lance data------
Passing st

In [9]:
players_with_data = [(i,j) for i, j in zip(df['player'], player_data)]

In [10]:
college_stats = pd.concat([i[1] for i in players_with_data])
college_stats['player'] = college_stats['name']
college_stats = college_stats.drop('name',axis=1)
df_with_college = pd.merge(df, college_stats, how='left', on='player')

In [11]:
df1 = df_with_college[df_with_college['seasons'] != 7.0]
df2 = df_with_college[df_with_college['seasons'] == 7.0]
df2['seasons'] = [3,4,3]
df_with_college = pd.concat([df1, df2])

/var/folders/k_/y5wzmsx1483f9z9t9tjtdj1m0000gn/T/ipykernel_2640/1477834817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['seasons'] = [3,4,3]


In [12]:
df_with_college.columns

Index(['player', 'successful', 'season', 'round', 'pick', 'team', 'age',
       'college', 'power_5', 'G', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%',
       'Int', 'Int%', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'seasons'],
      dtype='object')

In [13]:
df_with_college = df_with_college[['player', 'season', 'round', 'pick', 'team', 'age',
       'college', 'power_5', 'seasons','G', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%',
       'Int', 'Int%', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'successful']]

In [14]:
df_with_college.to_csv('first_round_qb_training_data.csv',index=False)

In [15]:
df_with_college.tail(10)

,player,season,round,pick,team,age,college,power_5,seasons,G,...,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,successful
33,Sam Darnold,2018,1,3,NYJ,21.0,USC,True,2.0,27,...,57,6.7,22,2.6,8.5,8.72,13.2,267.7,153.7,False
34,Josh Allen,2018,1,7,BUF,22.0,Wyoming,False,3.0,27,...,44,6.8,21,3.2,7.8,7.71,13.9,187.6,137.7,True
35,Josh Rosen,2018,1,10,ARI,21.0,UCLA,True,3.0,30,...,59,5.0,26,2.2,8.0,7.99,13.1,311.4,140.1,False
36,Lamar Jackson,2018,1,32,BAL,21.0,Louisville,True,3.0,38,...,69,6.4,27,2.5,8.3,8.48,14.6,238.0,142.9,True
37,Mitchell Trubisky,2017,1,2,CHI,23.0,North Carolina,True,3.0,34,...,41,7.2,10,1.7,8.3,8.97,12.3,140.1,157.6,False
38,Patrick Mahomes,2017,1,10,KAN,21.0,Texas Tech,True,3.0,32,...,93,6.9,29,2.1,8.3,8.75,13.1,351.6,152.0,True
39,Deshaun Watson,2017,1,12,HOU,21.0,Clemson,True,3.0,38,...,90,7.5,32,2.7,8.4,8.72,12.5,267.4,157.5,True
2,Cam Newton,2011,1,1,CAR,22.0,Auburn,True,3.0,6,...,0,0.0,0,0.0,4.5,4.50,9.0,9.0,87.8,True
25,Joe Burrow,2020,1,1,CIN,23.0,LSU,True,4.0,11,...,2,5.1,0,0.0,7.4,8.38,9.9,26.1,153.1,True
29,Kyler Murray,2019,1,1,ARI,22.0,Oklahoma,False,3.0,8,...,5,4.1,7,5.8,5.7,3.89,9.5,85.8,109.2,True


In [103]:
import warnings
pd.value_counts(df_with_college['college'])

/var/folders/k_/y5wzmsx1483f9z9t9tjtdj1m0000gn/T/ipykernel_90575/1295505125.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(df_with_college['college'])


college
Oklahoma            3
Florida St.         3
Ohio St.            2
Alabama             2
North Dakota St.    2
Oregon              2
Louisville          2
Clemson             2
Texas A&M           2
USC                 1
UCLA                1
North Carolina      1
Duke                1
Texas Tech          1
Utah St.            1
Auburn              1
Wyoming             1
Memphis             1
BYU                 1
Pittsburgh          1
Florida             1
California          1
Central Florida     1
Oklahoma St.        1
Baylor              1
Stanford            1
Missouri            1
Washington          1
LSU                 1
Name: count, dtype: int64

In [95]:
skimpy.skim(df_with_college)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 40     │ │ string      │ 17    │                                                          │
│ │ Number of columns │ 24     │ │ int64       │ 3     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 2     │                                                          │
│                                │ bool        │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name     ┃ NA   ┃ NA %   ┃ mean     ┃ sd      ┃ p0     ┃ p25    ┃ p50    ┃ p75    ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ season          │    0 │      0 │     2016 │   3.692 │   2010 │   2013 │   2017 │   2019 │  2022 │ ▅▃▃▅▆▇  │  │
│ │ round           │    0 │      0 │        1 │       0 │      1 │      1 │      1 │      1 │     1 │     ▇   │  │
│ │ pick            │    0 │      0 │      9.6 │   9.408 │      1 │      2 │    6.5 │     15 │    32 │ ▇▃▂ ▂▁  │  │
│ │ age             │    0 │      0 │    22.12 │   1.305 │     21 │     21 │     22 │     23 │    28 │   ▇▃▁   │  │
│ │ seasons         │    2 │      5 │    3.447 │   1.132 │      2 │      3 │      3 │      4 │     8 │  ▂▇▃▁   │  │
│ └─────────────────┴──────┴────────┴──────────┴─────────┴────────┴────────┴────────┴────────┴───────┴─────────┘  │
│                                                      bool                                                       │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                        ┃ true            ┃ true rate                     ┃ hist                ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ power_5                            │              31 │                          0.78 │       ▂    ▇        │  │
│ │ successful                         │              18 │                          0.45 │       ▇    ▇        │  │
│ └────────────────────────────────────┴─────────────────┴───────────────────────────────┴─────────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ player                    │       0 │          0 │                            2 │                       81 │  │
│ │ team                      │       0 │          0 │                            1 │                       40 │  │
│ │ college                   │       0 │          0 │                          1.4 │                       56 │  │
│ │ G                         │       2 │          5 │  